In [1]:
#Main Dependencies
import pandas as pd
import requests
from tkinter import *
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import time
import numpy as np

In [2]:
#Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/88.0.4324.96/chromedriver_win32.zip


[WDM] - Driver has been saved in cache [C:\Users\redea\.wdm\drivers\chromedriver\win32\88.0.4324.96]


In [ ]:
#Establish URL
url = 'https://coinmarketcap.com/'

In [ ]:
tokens = []

for i in range(0, 43, 1):
    loop_url = f'https://coinmarketcap.com/?page={i}'
    browser.visit(loop_url)
    
    for j in range(10):
        browser.execute_script("window.scrollBy(0,1000);")
        time.sleep(.5)
        
    html = browser.html
    
    soup = BeautifulSoup(html, 'html.parser')
    
    table = soup.find('table', class_='cmc-table')
    trs = table.find_all('tr')

    for tr in trs:

        token = {}

        tds = tr.find_all('td')

        try:
            if tds[2].p is not None:
                coin_name = tds[2].p.text
                coin_code = tds[2].div.div.div.p.text
            else:
                coin_name = tds[2].find_all('span')[1].text
                coin_code = tds[2].find_all('span')[2].text

            coin_price = float(tds[3].text.replace(',','').strip('$'))
            coin_24h = float(tds[4].text.strip('%'))
            coin_7d = float(tds[5].text.strip('%'))
            
            try:
                coin_market_cap = float(tds[6].text.replace(',','').strip('$'))
            except Exception as e:
                coin_market_cap = None
                print(f'Warning: row [{i}] is missing market cap.')                

            # from Kick: use this as an example to isolate volume, which is sometimes blank
            try:
                coin_volume_usd = float(tds[7].find_all('p')[0].text.replace(',','').strip('$'))
                coin_volume_crypto_list = tds[7].find_all('p')[1].text.replace(',','').strip('$').split()
                coin_volume = float(coin_volume_crypto_list[0])
                coin_volume_token = coin_volume_crypto_list[1]
            except Exception as e:
                i = trs.index(tr)
                print(f'Warning: row [{i}] is missing volume.')
                coin_volume_usd = None
                coin_volume_crypto_list = None
                coin_volume = None
                coin_volume_token = None

            try:
                coin_circulating_supply_list = tds[8].text.replace(',','').strip('$').split()
                coin_circulating_supply = float(coin_circulating_supply_list[0])
                coin_circulating_supply_token = coin_circulating_supply_list[1]
            except Exception as e:
                print(f'Warning: row [{i}] is missing circulating supply.')                
                coin_circulating_supply_list = None
                coin_circulating_supply = None
                coin_circulating_supply_token = None
                

            # set up dictionary record
            token['token'] = coin_code
            token['name'] = coin_name
            token['price'] = coin_price
            token['24h'] = coin_24h
            token['7d'] = coin_7d
            token['market_cap'] = coin_market_cap
            token['volume_usd'] = coin_volume_usd
            token['volume'] = coin_volume
            token['circulating_supply'] = coin_circulating_supply

            # add to the list of dictionaries
            tokens.append(token)

        except Exception as e:
            i = trs.index(tr)
            #print(f'Error with index [{i}] of the tr.')
            continue
            
browser.quit()

In [ ]:
#Create DataFrame
token = pd.DataFrame(tokens)
token

In [ ]:
#Extract Master array of NAMES ONLY for all Cryptos
coins = token['name'].values
#coins

In [ ]:
#Extract Master array of NAMES ONLY for all Cryptos
codes = token['token'].values
#codes
#len(codes)

<h1>Main Currency Query<h1>

In [ ]:
#Setting Up Drop Down Function
root = Tk()
root.geometry( "200x200" )
def show():
    label.config( text = clicked.get() )
    
options = coins
clicked = StringVar()
clicked.set('Click Me!')

drop = OptionMenu( root, clicked, *options)
drop.pack()

button = Button(root, text = 'Select This Coin?', command =show).pack()

label= Label(root, text = " ")
label.pack()

root.mainloop()

content = clicked.get()
#content
crypto = content
crypto

#Don't Forget to 'x' out of the prompt!

In [ ]:
code_filter = token[token['name']==crypto]
code_df = code_filter.values[0]
tick = code_df[0]
tick

In [ ]:
#FORMAT QUERY
lowercase = str.lower(crypto)
formed = lowercase.replace(" ","-")
#formed
exchange = f'https://www.coindesk.com/price/{formed}'
#exchange

<h1>Yahoo Historic Search DB</h1>

In [ ]:
#You can sub the second number in the range for len(codes)
history = []
for c in range(0,11,1):
    url = f'https://finance.yahoo.com/quote/{codes[c]}-USD/history?p={codes[c]}-USD'
    hist = requests.get(url)
    ho = BeautifulSoup(hist.text, 'html.parser')
    try:
        tabbed = pd.read_html(url)[0]
    except:
        continue
    tabbed['Coin'] = codes[c]
    history.append(tabbed)

    
yahoo = pd.concat(history)
#yahoo

In [ ]:
#Export to CSV
yahoo.to_csv('History.csv', index=False, header=True)

In [ ]:
#Export to JSON
import csv
import json

def make_json(csvFilePath, jsonFilePath):

    data = {}
     
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
         
        for rows in csvReader:

            key = rows['Coin']
            data[key] = rows

    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))
         
csvFilePath = r'History.csv'
jsonFilePath = r'History.json'
 
make_json(csvFilePath, jsonFilePath)

<h1>API History Pull - Single Search</h1>

<href>https://nomics.com/docs/#section/Introduction</href>

In [ ]:
import json
API = '40925fa28c8e33ae59c34f12791f07d6'

In [ ]:
start = input('What date would you like to start at (yyyy-mm-dd)? ')
end = input('What date would you like to end at (yyyy-mm-dd)? ')

In [ ]:
volhist_url = f'https://api.nomics.com/v1/volume/history?key={API}&exchange={formed}&start={start}T00%3A00%3A00Z&end={end}T00%3A00%3A00Z'
marketcap_url = f'https://api.nomics.com/v1/market-cap/history?key={API}&start={start}T00%3A00%3A00Z&end={end}T00%3A00%3A00Z'
exchangerate_url = f'https://api.nomics.com/v1/exchange-rates/history?key={API}&currency={tick}&start={start}T00%3A00%3A00Z&end={end}T00%3A00%3A00Z'

In [ ]:
#Volume History
response = requests.get(volhist_url)
data = response.json()
print(json.dumps(data, indent=4, sort_keys=True))

In [ ]:
#Market Cap
response = requests.get(marketcap_url)
data = response.json()
print(json.dumps(data, indent=4, sort_keys=True))

In [ ]:
#Exchange Rate
response = requests.get(exchangerate_url)
data = response.json()
print(json.dumps(data, indent=4, sort_keys=True))

<h1>Exchange Information</h1>

In [ ]:
#NEW URL
stats = f'https://coinmarketcap.com/currencies/{formed}/'

In [ ]:
#Cooking in the Kitchen
analysis = requests.get(stats)
broth = BeautifulSoup(analysis.text, 'html.parser')
#broth

In [ ]:
#Pulling Tables in HTML
tables = broth.findAll('table')
#tables

In [ ]:
#First Table
first_table_scrape = tables[0]
# first_table_scrape
first_table_tds = first_table_scrape.find_all('td')
#first_table_tds

In [ ]:
#Second Table
second_table_scrape = tables[1]
#second_table_scrape
second_table_tds = second_table_scrape.find_all('td')
#second_table_tds

In [ ]:
#Third Table
third_table_scrape = tables[2]
#third_table_scrape
third_table_tds = third_table_scrape.find_all('td')
#third_table_tds

In [ ]:
#Fourth Table
fourth_table_scrape = tables[3]
#fourth_table_scrape
fourth_table_tds = fourth_table_scrape.find_all('td')
#fourth_table_tds

In [ ]:
#Fifth Table
fifth_table_scrape = tables[4]
#fifth_table_scrape
fifth_table_tds = fifth_table_scrape.find_all('td')
#fifth_table_tds

In [ ]:
#Pull ABOUT Narrative
texts = broth.select('div.about___1OuKY')
#texts

In [ ]:
#Token Pull
code = broth.findAll('div', class_='sc-16r8icm-0 fCTazK nameHeader___27HU_')[0].find_all('small')[0].getText()

In [ ]:
#Establishing List
tables = []

#Creating the Dictionary
table = {}

#First Table
price = first_table_tds[0].get_text()
pricechange24hr = first_table_tds[1].find_all('span')[0].get_text()
percentchange24hr = first_table_tds[1].find_all('span')[1].get_text()
low24hr = first_table_tds[2].find_all('div')[0].get_text().split(' /')[0]
high24hr = first_table_tds[2].find_all('div')[1].get_text().split(' /')[0]
tradingvolume = first_table_tds[3].find_all('span')[0].get_text()
tradingvolumepercdelta = first_table_tds[3].find_all('span')[1].get_text()
marketdominance = first_table_tds[4].find_all('span')[0].get_text()
marketrank = first_table_tds[5].get_text().split('#')[1]

#Second Table
marketcap = second_table_tds[0].find_all('span')[0].get_text()
fullydiluted = second_table_tds[1].find_all('span')[0].get_text()

#Third Table
yestlow = third_table_tds[0].find_all('div')[0].get_text().split(' /')[0]
yesthigh = third_table_tds[0].find_all('div')[1].get_text().split(' /')[0]
yestopen = third_table_tds[1].find_all('div')[0].get_text().split(' /')[0]
yestclose = third_table_tds[1].find_all('div')[1].get_text().split(' /')[0]
yestvol = third_table_tds[3].get_text()

#Fourth Table
d7low = fourth_table_tds[0].find_all('div')[0].get_text().split(' /')[0]
d7high = fourth_table_tds[0].find_all('div')[1].get_text().split(' /')[0]
d30low = fourth_table_tds[1].find_all('div')[0].get_text().split(' /')[0]
d30high = fourth_table_tds[1].find_all('div')[1].get_text().split(' /')[0]
d90low = fourth_table_tds[2].find_all('div')[0].get_text().split(' /')[0]
d90high = fourth_table_tds[2].find_all('div')[1].get_text().split(' /')[0]
wk52low = fourth_table_tds[3].find_all('div')[0].get_text().split(' /')[0]
wk52high = fourth_table_tds[3].find_all('div')[1].get_text().split(' /')[0]
alltimehigh = fourth_table_tds[4].find_all('span')[0].get_text()
alltimehighperc = fourth_table_tds[4].find_all('span')[1].get_text()
alltimelow = fourth_table_tds[5].find_all('span')[0].get_text()
alltimelowperc = fourth_table_tds[5].find_all('span')[1].get_text()
roi = fourth_table_tds[6].findAll('p')[0].getText()

#Fifth Table
circsupp = fifth_table_tds[0].get_text()
totsupp = fifth_table_tds[1].get_text()
mxsupp = fifth_table_tds[2].get_text()

#Narrative
intro1 = texts[1].find_all('p')[0].get_text()
intro2 = texts[1].find_all('p')[1].get_text()
fullintro = f'{intro1}{intro2}'

#Building the Dictionary
table['Code'] = code
table['Price'] = price
table['24hr Price Change'] = pricechange24hr
table['24hr % Change'] = percentchange24hr
table['24hr Low'] = low24hr
table['24hr High'] = high24hr
table['24hr Trading Volume'] = tradingvolume
table['24hr Trading Volume % Change'] = tradingvolumepercdelta
table['Market Dominance'] = marketdominance
table['Market Rank'] = marketrank
table['Market Cap'] = marketcap
table['Fully Diluted Market Cap'] = fullydiluted
table['Yesterday Low'] = yestlow
table['Yesterday High'] = yesthigh
table['Yesterday Open'] = yestopen
table['Yesterday Close'] = yestclose
table['Yesterday Volume'] = yestvol
table['7 Day Low'] = d7low
table['7 Day High'] = d7high
table['30 Day Low'] = d30low
table['30 Day High'] = d30high
table['90 Day Low'] = d90low
table['90 Day High'] = d90high
table['52 Week Low'] = wk52low
table['52 Week High'] = wk52high
table['All Time High'] = alltimehigh
table['All Time High %'] = alltimehighperc
table['All Time Low'] = alltimelow
table['All Time Low %'] = alltimelowperc
table['Return on Investment'] = roi
table['Cirulating Supply'] = circsupp
table['Total Supply'] = totsupp
table['Max Supply'] = mxsupp
table["About"] = fullintro
table['Link'] = stats

#Appending
tables.append(table)

#Print Final Dictionary
tables

In [ ]:
#LOGO PHOTO
lc_code = str.lower(code)
snap = f'https://cryptologos.cc/logos/{formed}-{lc_code}-logo.png?v=010'
print(snap)

<h1>Market Purchase Information</h1>

In [ ]:
#NEW URL
markets = f'https://coinmarketcap.com/currencies/{formed}/markets/'

In [ ]:
#Cooking in the Kitchen
whereto = requests.get(markets)
bisque = BeautifulSoup(whereto.text, 'html.parser')
#bisque

In [ ]:
#Pulling Tables in HTML
frames = bisque.findAll('table')
board = frames[0]
#board

In [ ]:
#Setting up trs -- remember to skip [0] tr since it's a header
board_trs = board.find_all('tr')
#board_trs

In [ ]:
#Top 100 Crypto Markets for Query
guides = []

for i in range(1,101,1):
    
    tds = board_trs[i].find_all('td')

    guide = {}

    source = tds[1].find_all('p')[0].getText()
    pairs = tds[2].find_all('a')[0].getText()
    web = tds[2].find_all('a')[0].get('href')
    price = tds[3].get_text()
    volume = tds[4].find_all('p')[0].getText()
    volumeperc = tds[5].find_all('div')[0].get_text()
    liquid = tds[6].find_all('p')[0].getText()
    confidence = tds[7].find_all('div')[0].get_text()
    updated = tds[8].find_all('p')[0].getText()

    guide['Source'] = source
    guide['Currency'] = pairs
    guide['URL'] = web
    guide['Price'] = price
    guide['Volume'] = volume
    guide['Volume %'] = volumeperc
    guide['Liquidity'] = liquid
    guide['Confidence'] = confidence
    guide['Last Updated'] = updated

    guides.append(guide)

#guides
blackmarket = pd.DataFrame(guides)
blackmarket

<h1>Historical Data</h1>

In [ ]:
#NEW URL - Creating Date List
dates = 'https://coinmarketcap.com/historical/'

In [ ]:
#Cooking in the Kitchen
select = requests.get(dates)
pho = BeautifulSoup(select.text, 'html.parser')
#pho

In [ ]:
#Pull history of all dates
links = pho.find_all('div',class_='cmc-bottom-margin-2x')[0].find_all('a',class_='historical-link cmc-link')
#links

In [ ]:
#Create DataFrame
dates = []

for l in range(36,len(links),1):
    
    date = {}
    
    nums=links[l].get('href').split("/")[2]
    
    date['Raw'] = nums

    dates.append(date)
    
timeline = pd.DataFrame(dates)
#timeline

In [ ]:
#Reformat DataFrame
timeline['Form'] = timeline['Raw'].astype(str)
timeline['Day'] = timeline['Form'].str[6:8]
timeline['Month'] = timeline['Form'].str[4:6]
timeline['Year'] = timeline['Form'].str[0:4]
timeline.drop('Form', axis=1, inplace=True)
#timeline

In [ ]:
#Add the Query column to the DataFrame
timeline["Query"] = timeline["Month"] + "/" + timeline["Day"] + "/" + timeline["Year"]
#timeline
specs = timeline['Query'].values
#specs

In [ ]:
#Create a Date select function
root = Tk()
root.geometry( "200x200" )
def show():
    label.config( text = clicked.get() )
    
options = specs
clicked = StringVar()
clicked.set('Click Me!')

drop = OptionMenu( root, clicked, *options)
drop.pack()

button = Button(root, text = 'Select This Date?', command =show).pack()

label= Label(root, text = " ")
label.pack()

root.mainloop()

content = clicked.get()
#content
birth = content
birth

#Don't Forget to 'x' out of the prompt!

In [ ]:
#Reformat the User Input for URL variable
reform = birth.split('/')
history_input = f'{reform[2]}{reform[0]}{reform[1]}'
#history_input

In [ ]:
#NEW URL
history = f'https://coinmarketcap.com/historical/{history_input}/'
#print(history)

In [ ]:
#Cooking in the Kitchen
story = requests.get(history)
chowder = BeautifulSoup(story.text, 'html.parser')
#chowder

In [ ]:
#Dive into the table
exhibits = chowder.findAll('tbody')
itinerary = exhibits[0]
#itinerary

In [ ]:
#Locate table data
itinerary_trs = itinerary.find_all('tr')
#itinerary_trs

In [ ]:
#Max of Top 200 Crypto Histories
logs = []

for w in range(0,len(itinerary),1):
    
    tds = itinerary_trs[w].find_all('td')

    log = {}

    rank = tds[0].find_all('div')[0].get_text()
    name = tds[1].find_all('a')[0].getText()
    symbol = tds[2].find_all('div')[0].get_text()
    marketcap = tds[3].find_all('p')[0].getText()
    price = tds[4].find_all('a')[0].getText()
    circsupp = tds[5].find_all('div')[0].get_text()
    vol24h = tds[6].find_all('a')[0].getText()
    p1h = tds[7].find_all('div')[0].get_text()
    p24h = tds[8].find_all('div')[0].get_text()
    p7d = tds[9].find_all('div')[0].get_text()

    log['Rank'] = rank
    log['Name'] = name
    log['Symbol'] = symbol
    log['Market Cap'] = marketcap
    log['Price'] = price
    log['Circulating Supply'] = circsupp
    log['1 Hour Percent Change'] = p1h
    log['24 Hour Percent Change'] = p24h
    log['7 Day Percent Change'] = p7d
    

    logs.append(log)

#logs
print(f'-------------Historical Snapshot for {formed} on {birth}-------------')
textbook = pd.DataFrame(logs)
textbook.head()

<h1>Tweepy</h1>

In [ ]:
#SHOULD BE PULLED DAILY TO NOT OVERLOAD API REQUESTS
!pip install tweepy
import tweepy
import json
from config import Consumer_Key, Consumer_Secret, Access_Token, Access_Token_Secret

In [ ]:
#Establishing API - REMEMBER TO CHANGE SEARCH TERM
auth = tweepy.OAuthHandler(Consumer_Key,Consumer_Secret)
auth.set_access_token(Access_Token,Access_Token_Secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())
#30 Day Analysis
tokes = api.search_30_day('month',crypto)
#tokes

In [ ]:
#Testing Pull
results = tokes['results']
#len(results)
#results

In [ ]:
#30 Day DataFrame
tweets = []

for r in results:
    
    tweet = {}
    

    dates = r['created_at'].split(" ")
    date = dates[0] + "., " + dates[1] + ". " + dates[2]
    text = r['text']
    location = r['user']['location']
    description = r['user']['description']
    favs = r['user']['favourites_count']
    screen = r['user']['screen_name']
    
    tweet['Screen Name'] = f'@{screen}'
    tweet['Date'] = date
    tweet['Text'] = text
    tweet['Location'] = location
    tweet['Description'] = description
    tweet['Favorited'] = favs
    
    tweets.append(tweet)

#tweets
bird = pd.DataFrame(tweets)
bird.head()

In [ ]:
#Full Search
full = api.search(crypto)
#full

In [ ]:
#Testing Pull
series = full['statuses']
#len(series)
#series

In [ ]:
#Full Search DataFrame
steps = []

for s in series:
    
    step = {}
    

    dates = s['created_at'].split(" ")
    date = dates[0] + "., " + dates[1] + ". " + dates[2]
    text = s['text']
    location = s['user']['location']
    description = s['user']['description']
    favs = s['user']['favourites_count']
    screen = s['user']['screen_name']
    
    step['Screen Name'] = f'@{screen}'
    step['Date'] = date
    step['Text'] = text
    step['Location'] = location
    step['Description'] = description
    step['Favorited'] = favs
    
    steps.append(step)

#steps
wing = pd.DataFrame(steps)
wing.head()

<h1>Reddit</h1>

In [ ]:
#Establish Python Reddit Wrapper
!pip install praw
import praw
from config import Client_ID, Client_Secret, User_Agent

In [ ]:
#Connect Credentials
reddit = praw.Reddit(client_id=Client_ID, client_secret=Client_Secret, user_agent=User_Agent)

In [ ]:
#Pull Top 50 Posts for Query SubReddit
posts = []
try:
    crypt_read = reddit.subreddit(formed)
except Exception as e:
    print('Sorry! That Crypto does not have a SubReddit. Redirecting you to the general CrytoCurrency SubReddit.')
    crypt_read = reddit.subreddit('cryptocurrency')
    
for post in crypt_read.hot(limit=50):
    posts.append([post.title, post.score, post.subreddit, post.url, post.num_comments])
    karma = pd.DataFrame(posts,columns=['Title','Score','SubReddit','URL','# of Comments'])
        
karma.head()

<h1>General Media - Top News Stories</h1>

In [ ]:
media = 'https://www.coindesk.com/category/markets'

In [ ]:
journal = requests.get(media)
eggdrop = BeautifulSoup(journal.text, 'html.parser')
#eggdrop

In [ ]:
dumpling = eggdrop.find('div',class_='page-area-dotted').find('section',class_='page-area-dotted-content').find('section',class_='list-body')
#dumpling

In [ ]:
#Top 9 News Stories from CoinDesk -- MAYBE PUT THIS IN A TICKER FOR VISUAL?
articles = []

for a in range(0,len(dumpling),1):

    article = {}

    title = dumpling.find_all('div',class_='text-content')[a].find_all('h4')[0].getText()
    link = dumpling.find_all('div',class_='text-content')[a].find_all('a')[1].get('href')
    description = dumpling.find_all('div',class_='text-content')[a].find_all('p')[0].getText()

    article['Title'] = title
    article['Link'] = f'https://www.coindesk.com{link}'
    article['Description'] = description

    articles.append(article)

#articles
newspaper = pd.DataFrame(articles)
newspaper

<h1>101 Guides to CrytoCurrency (CoinDesk)</h1>

In [ ]:
classroom = 'https://www.coindesk.com/learn'

In [ ]:
desk = requests.get(classroom)
chickenoodle = BeautifulSoup(desk.text, 'html.parser')
#chickenoodle

In [ ]:
noodle = chickenoodle.find('div',class_='page-area-dotted').find('section',class_='page-area-dotted-content').find('section',class_='list-body')
#noodle

In [ ]:
#101 Guides - Count of 50
books = []

for b in range(0,len(noodle),1):

    book = {}

    title = noodle.find_all('div',class_='text-content')[b].find_all('h4')[0].getText()
    link = noodle.find_all('div',class_='text-content')[b].find_all('a')[1].get('href')
    description = noodle.find_all('div',class_='text-content')[b].find_all('p')[0].getText()

    book['Title'] = title
    book['Link'] = f'https://www.coindesk.com{link}'
    book['Description'] = description

    books.append(book)

#books
library = pd.DataFrame(books)
library.head()

<h1>Cryto Legality Around the Globe</h1>

In [3]:
dora = 'https://en.wikipedia.org/wiki/Legality_of_bitcoin_by_country_or_territory'

In [4]:
backpack = requests.get(dora)
ramen = BeautifulSoup(backpack.text, 'html.parser')
#ramen

In [5]:
#Pulling Tables in HTML
bowls = ramen.findAll('table')
#bowls
#len(bowls)

In [6]:
#Establishing List
rules = []

for bowl in range(3,len(bowls),1):
    
    head = bowls[bowl].find_all('tr')
    
    for x in range(1,len(head),1):

        #Creating the Dictionary
        rule = {}
        
        try:
            
            title = head[x].find_all('td')[0].find_all('a')[0].getText()
            legal = head[x].find_all('td')[1].get_text().split('\n')[0].split(' ')[1]
            body = head[x].find_all('td')[1].find_all('p')[0].getText().split('\n')[0]
        
        except:
            continue

        #Building the Dictionary
        rule['Country'] = title
        rule['Legal Status'] = legal
        rule['Blurb'] = body

        #Appending
        rules.append(rule)

#Print Final Data
#rules
atlantis = pd.DataFrame(rules)
atlantis

,Country,Legal Status,Blurb
0,Algeria,Illegal,"According to the ""Journal Officiel"" (28 Decemb..."
1,Egypt,Illegal,"""Egypt’s Dar al-Ifta, the primary Islamic legi..."
2,Morocco,Illegal,On 20 November 2017 the exchange office issued...
3,Nigeria,Legal,"As of 17 January 2017, The Central Bank of Nig..."
4,Mauritius,Legal,The Financial Services Commission of Mauritius...
...,...,...,...
80,Luxembourg,Legal,The Commission de Surveillance du Secteur Fina...
81,Netherlands,Legal,"As of 2017[update], virtual currencies such as..."
82,United Kingdom,Legal,"As of 2017[update], the government of the Unit..."
83,Australia,Legal,"In December 2013, the governor of the Reserve ..."


In [7]:
atlantis.to_csv("Countries.csv", index=False, header=True)

<h1>BitCoin Depot</h1>

In [ ]:
#Import API
from config import Google_API

In [ ]:
bee = 'https://bitcoindepot.com/locations/'

In [ ]:
#Reading URL
gold = requests.get(bee)
mapo = BeautifulSoup(gold.text, 'html.parser')
#mapo

In [ ]:
#Pulling Addresses in HTML
tofu = mapo.findAll('div',class_='list-country-card')
#tofu
#len(tofu)

In [ ]:
#Establishing List
atms = []

for t in range(0,42,1):
    
    addresses = tofu[t].findAll('span',class_='list-country-list-text')
    states = tofu[t].findAll('a',class_='get-direction-link')
    hours = tofu[t].findAll('span',class_='list-country-list-time')

    for a in range(0,len(addresses),1):
        
        atm = {}
        
        ave = addresses[a].getText().split(', ')[2].split(' ')
        s = ' '
        street = s.join(ave)
        name = addresses[a].getText().split(', ')[1]
    
        for s in range(0,len(states),1):

            atm = {}
        
            raw = states[s].get('href').split('+')
            pull = len(raw)
            state = raw[pull-2]
            zipcode = raw[pull-1]
            city = raw[pull-3]

        for h in range(0,len(hours),1):

            hour = hours[h].getText().replace('\r\n',', ')
            
            atm['Name'] = name
            atm['Hours'] = hour
            atm['Address'] = f'{street} {city}, {state}, {zipcode}'
            atm['Street'] = street.split(',')[0]
            atm['City'] = city
            atm['State'] = state
            atm['Zip Code'] = zipcode

        atms.append(atm)
#atms
bitcoin = pd.DataFrame(atms)
bitcoin

In [ ]:
#Adding Columns
bitcoin['Latitude'] = ''
bitcoin['Longitude'] = ''
bitcoin

In [ ]:
#Geocode -- this takes a long time to run
params = {'key': Google_API}
for index, row in bitcoin.iterrows():
    base_url = 'https://maps.googleapis.com/maps/api/geocode/json'
    
    street = row['Street']
    city = row['City']
    state = row['State']
    zipcode = row['Zip Code']
    
    params['address'] = f'{street},{city},{state},{zipcode}'
    
    depots_lat_lng = requests.get(base_url, params=params)
    depots_lat_lng = depots_lat_lng.json()
    
    try:
        bitcoin.loc[index, 'Latitude'] = depots_lat_lng['results'][0]['geometry']['location']['lat']
        bitcoin.loc[index, 'Longitude'] = depots_lat_lng['results'][0]['geometry']['location']['lng']
    except:
        continue

In [ ]:
#Finalize All Data
bitcoin

In [ ]:
#Export as CSV
bitcoin.to_csv("ATMs.csv", index=False, header=True)

In [5]:
#Export as JSON
import csv
import json

def make_json(csvFilePath, jsonFilePath):

    data = {}
     
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
         
        for rows in csvReader:

            key = rows['Name']
            data[key] = rows

    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))
         
csvFilePath = r'ATMs.csv'
jsonFilePath = r'ATMs.json'
 
make_json(csvFilePath, jsonFilePath)